In [1]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim=1024):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model and load weights
model = MLP(input_dim=1024)
model.load_state_dict(torch.load("face_match_model.pth", map_location=device))
model.to(device)
model.eval()  # Set to evaluation mode


MLP(
  (net): Sequential(
    (0): Linear(in_features=1024, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [3]:
from keras_facenet import FaceNet
import cv2
import numpy as np

embedder = FaceNet()
img = cv2.imread(r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250525_11_28_48_Pro.jpg")
img2=cv2.imread(r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250601_21_15_45_Pro.jpg")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_rgb2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

embedding1 = embedder.embeddings([img_rgb])
embedding2= embedder.embeddings([img_rgb2])


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step


In [ ]:

emb1 = embedding1[0]  # shape (512,)
emb2 = embedding2[0]  # shape (512,)

# Concatenate the two embeddings
combined = np.concatenate([emb1, emb2])  # shape (1024,)

# Convert to PyTorch tensor and add batch dimension
input_tensor = torch.tensor(combined, dtype=torch.float32).unsqueeze(0).to(device)  # shape (1, 1024)

# Run through your loaded model
model.eval()
with torch.no_grad():
    score = model(input_tensor).item()

# Print result
print(f"Similarity Score: {score:.4f}")
if score > 0.5:
    print(" Same person")
else:
    print(" Different people")


Similarity Score: 0.2112
❌ Different people


In [4]:
def is_same_person(img_path1, img_path2, threshold=0.5):
    # Read and preprocess images
    img1 = cv2.imread(img_path1)
    img2 = cv2.imread(img_path2)
    if img1 is None or img2 is None:
        raise FileNotFoundError("One of the image paths is invalid or the image could not be loaded.")

    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    # Extract embeddings
    emb1 = embedder.embeddings([img1_rgb])[0]
    emb2 = embedder.embeddings([img2_rgb])[0]

    # Combine embeddings and prepare input tensor
    combined = np.concatenate([emb1, emb2])
    input_tensor = torch.tensor(combined, dtype=torch.float32).unsqueeze(0).to(device)

    # Get prediction
    with torch.no_grad():
        score = model(input_tensor).item()

    # Return result
    return score, score > threshold


In [8]:
score, is_match = is_same_person(
    r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250601_18_19_16_Pro.jpg",
    r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250601_21_15_45_Pro.jpg"
)

print(f"Similarity Score: {score:.4f}")
if is_match:
    print("Same person")
else:
    print("Different people")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Similarity Score: 0.2431
Different people
